<a href="https://colab.research.google.com/github/monyau/convert-subtitles-from-chs-to-cht/blob/master/convert_subtitles_chs2cht_V1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## 1. Mount Google Drive

from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
## 2. install applications

"""
##!pip install hanzidentifier   
hanziidentifier is not functioning properly
"""
!wget "https://github.com/monyau/convert-subtitles-from-chs-to-cht/blob/master/%E7%B0%A1%E9%AB%94%E5%AD%97_sim_dict.txt"
!apt -y install python3 python3-pip
!pip install opencc-python-reimplemented

In [ ]:
## 3. run script

# written by Monya
# -*- coding: utf-8 -*-
#  convert all subdirectories subtitles from chs to cht with openCC


import os, sys
import io, time
from chardet.universaldetector import UniversalDetector
import re
from opencc import OpenCC

global content_chs
file2 = open('/content/簡體字_sim_dict.txt' ,'r',encoding='utf8', errors="surrogateescape")
content_chs = file2.read()
file2.close()

Exit = False
inputPath = ""
while inputPath == "":
    inputPath = input('input a directory/path')
if not os.path.exists(inputPath):
    print('"'+inputPath+'"', 'does not exist.')
    Exit = True
      
"""
### Conversions 轉換

* `hk2s`: Traditional Chinese (Hong Kong standard) to Simplified Chinese

* `s2hk`: Simplified Chinese to Traditional Chinese (Hong Kong standard)

* `s2t`: Simplified Chinese to Traditional Chinese

* `s2tw`: Simplified Chinese to Traditional Chinese (Taiwan standard)

* `s2twp`: Simplified Chinese to Traditional Chinese (Taiwan standard, with phrases)

* `t2hk`: Traditional Chinese to Traditional Chinese (Hong Kong standard)

* `t2s`: Traditional Chinese to Simplified Chinese

* `t2tw`: Traditional Chinese to Traditional Chinese (Taiwan standard)

* `tw2s`: Traditional Chinese (Taiwan standard) to Simplified Chinese

* `tw2sp`: Traditional Chinese (Taiwan standard) to Simplified Chinese (with phrases)
"""



def detect_encoding(file_path):
    # Detect file encoding
    detector = UniversalDetector()
    detector.reset()
    with open(file_path, mode='rb') as f:
        for b in f:
            detector.feed(b)
            if detector.done: break
    detector.close()
    return detector.result


def detect_chs(content1):
    # detect chinese characters in ass/srt files
    Len_content1 = len(content1)
    Quit = False
    if re.search(u"[\u4e00-\u9FFF]", content1):  # all cht+chs chinese, Jap and Kor
        n = 0
        m = 0
        while n < Len_content1 and not Quit:
            chr = content1[n:n+1]
            if re.search(u"[\u4e00-\u9FFF]", chr):  # all cht+chs chinese, Jap and Kor
                match = re.search(chr, content_chs)
                if match:
                    m = m + 1
                    print(chr, end="")  # print without newline
                    if m >= 30:
                        Quit = True
            n = n + 1
        if m >= 30:
            #print('{}{}'.format(subindent, InputFileName)+ " -> is chs")
            return 'chs'
        else:
            #print('{}{}'.format(subindent, InputFileName)+ " -> is cht")
            return 'cht'

    else:
        #print('{}{}'.format(subindent, InputFileName)+ " -> is not chinese")
        return 'not Chinese'


def convertFile(inputPath_sub, InputFileName1, result_T):

    outputPath_cht =  inputPath_sub + "/utf8-cht"
    cc_s2t = OpenCC('s2twp')  # convert from Simplified Chinese to Traditional Chinese

    file_encoding = detect_encoding(inputPath_sub + '/' + InputFileName1)
    file_encoding = str(file_encoding['encoding'])

    content1 = ""   # init
    if 'gb2312' in file_encoding.lower() or 'gb18030' in file_encoding.lower() or 'cp950' in file_encoding.lower() or 'gbk' in file_encoding.lower() or 'hz' in file_encoding.lower():
        file1 = io.open(inputPath_sub + '/' + InputFileName1, 'r', encoding = 'gb18030')
    else:    
        file1 = io.open(inputPath_sub + '/' +InputFileName1, 'r', encoding = file_encoding)
    content1 = file1.read() 
    file1.close()

    CH = detect_chs(content1)
    if CH == 'chs':
        file2 = (outputPath_cht + '/' +result_T)
        if os.path.exists(file2):
            if os.stat(file2).st_size == 0:  # file is empty
                #os.remove(file2)    # do not work in colab!!
                print(file2, ' is empty.')
            else:
                print('{}{}'.format(subindent, result_T)+ " -> is already existed") ##########
        else:
            print('{}{}'.format(subindent, InputFileName1)+ " -> is "+ file_encoding+ ", "+ CH) ##########
    else:
            print('{}{}'.format(subindent, InputFileName1)+ " -> is "+ file_encoding+ ", "+ CH) ##########

    if CH == "chs" and (not os.path.exists(file2)) and (InputFileName.lower().find(".cht.") < 0):
        if not os.path.exists(outputPath_cht):  # make target subpath
            os.makedirs(outputPath_cht)
        file2 = open(outputPath_cht + '/' +result_T, 'w', encoding = 'utf-8', errors="surrogateescape")

        content2 = cc_s2t.convert(content1)
        content2 = content2.replace(u'几',u'幾').replace(u'释',u'釋')
        time.sleep(1)

        file2.write(content2)
            
        time.sleep(1)    
        file2.close()

        print("Subtitle: "+'{}{}'.format(subindent, InputFileName1)+ " -> convert to cht")  # files in subpath


if not Exit:
    for root, dirs, files in os.walk(inputPath):
        dirs.sort()
        level = root.replace(inputPath, '').count(os.sep)
        inputPath_sub = root   # the full path
        indent = ' ' * 4 * (level)  # left with 4 white spaces

        if 'utf8-cht' in os.path.basename(root):
            Ouit = True
        else:
            Quit = False

        print("path: "+ '{}{}/'.format(indent, os.path.basename(root)))  # not the full subpath
    
        subindent = ' ' * 4 * (level + 1)
        files.sort()
        Quit = False
        for f in files:
            InputFileName = f[f.rfind("/")+1:]        #file name without path
            if os.path.isdir(InputFileName):
                pass
            else:
                filename = os.path.join(root, f)  # file name with path
                fileSize = os.stat(filename).st_size
                if fileSize == 0:
                    Quit = Ture
                if not Quit and (InputFileName.lower().endswith(".srt") or InputFileName.lower().endswith(".ass")) :
                    if not InputFileName.lower().find(".cht.") > 0:
                        if InputFileName.lower().find(".chs.") > 0:   # replace ".chs." to ".cht."
                            result_T = InputFileName[0:(InputFileName.lower().find(".chs."))]+".cht."+InputFileName[(InputFileName.lower().find(".chs."))+5:]
                        elif InputFileName.lower().endswith(".srt") and InputFileName.lower().find(".chs.") < 0 and InputFileName.lower().find(".cht.") < 0:
                            result_T = InputFileName[0:-3] + "cht.srt"
                        elif InputFileName.lower().endswith(".ass") and InputFileName.lower().find(".chs.") < 0 and InputFileName.lower().find(".cht.") < 0:
                            result_S = InputFileName[0:-3] + "chs.ass"
                        else:
                            result_T = InputFileName
                    
                        time.sleep(1)
                        convertFile(inputPath_sub, InputFileName,result_T)
                    


